# Retail Sales Forecasting Project
Forecast weekly sales using ARIMA and Prophet.

In [ ]:
# 1. Import libraries
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error

plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# 2. Load dataset
df = pd.read_csv('retail_sales.csv', parse_dates=['date'])
df.set_index('date', inplace=True)
df = df.asfreq('W')  # Weekly frequency
df['sales'].interpolate(inplace=True)
df.head()

In [ ]:
# 3. EDA
df['sales'].plot(title='Weekly Sales')
plt.show()

In [ ]:
# 4. ARIMA Model
train = df.iloc[:-12]
test = df.iloc[-12:]
model_arima = ARIMA(train['sales'], order=(1,1,1)).fit()
forecast_arima = model_arima.forecast(steps=12)
print('ARIMA MAE:', mean_absolute_error(test['sales'], forecast_arima))

In [ ]:
# 5. Prophet Model
prophet_df = df.reset_index()[['date','sales']].rename(columns={'date':'ds','sales':'y'})
m = Prophet()
m.fit(prophet_df.iloc[:-12])
future = m.make_future_dataframe(periods=12, freq='W')
forecast = m.predict(future)
prophet_forecast = forecast.set_index('ds')['yhat'][-12:]
print('Prophet MAE:', mean_absolute_error(test['sales'], prophet_forecast))

In [ ]:
# 6. Compare and Plot
plt.plot(test.index, test['sales'], label='Actual')
plt.plot(test.index, forecast_arima, label='ARIMA Forecast')
plt.plot(test.index, prophet_forecast, label='Prophet Forecast')
plt.legend(); plt.title('Model Comparison'); plt.show()

In [ ]:
# 7. Save Forecasts
output = pd.DataFrame({
    'actual': test['sales'],
    'arima_forecast': forecast_arima,
    'prophet_forecast': prophet_forecast
})
output.to_csv('sales_forecast_comparison.csv', index=True)
print('Forecast saved to sales_forecast_comparison.csv')